## Build Time - Tool Enrichment Example

Demonstrates using Tool Enrichment from ALTK to perform enrichment of python tool docstring using the metadata information in the tool definition for improved tool calling and tool input formation.

#### Python Tool:

In [1]:
import os

os.environ["WX_API_KEY"] = "xxxx"
os.environ["WX_PROJECT_ID"] = "xxxx"

In [2]:
python_tool = '''
from enum import Enum
from typing import ClassVar, Optional, Type
import datetime
from dataclasses import dataclass
from langchain_core.tools import tool

class TimeOffTypes(Enum):
    """Represents the time off event types"""

    ABSENCE = "ABSENCE"
    PUBLIC_HOLIDAY = "PUBLIC_HOLIDAY"
    NON_WORKING_DAY = "NON_WORKING_DAY"

@dataclass
class UpcomingTimeOff:
    """Represents an upcoming time off event"""

    title: str
    start_date: str
    end_date: str
    start_time: Optional[str]
    end_time: Optional[str]
    duration: int
    time_unit: str
    cross_midnight: bool
    type: str
    status_formatted: Optional[str]
    absence_duration_category: Optional[str]

    #Schema: ClassVar[Type[Schema]] = Schema

@dataclass
class UpcomingTimeOffResponse:
    """Represents the response from getting a user's upcoming time off."""

    time_off_events: list[UpcomingTimeOff]

    #Schema: ClassVar[Type[Schema]] = Schema


@tool
def get_upcoming_time_off(
    user_id: str, start_date: str, end_date: str, time_off_types: list[str]
) -> UpcomingTimeOffResponse:
    """
    Retrieves the user's upcoming time off details from SAP SuccessFactors.

    """

    # Check for date format
    try:
        datetime.datetime.strptime(start_date, "%Y-%m-%d")
        datetime.datetime.strptime(end_date, "%Y-%m-%d")
    except ValueError:
        return {"error": "Invalid date format. Please use YYYY-MM-DD."}

    time_off_events=[]

    for  time_type in time_off_types:
    # Hard-coded values for testing
        if time_type == "ABSENCE":
            time_off_events.append(
                UpcomingTimeOff(
                    title="Vacation",
                    start_date="2024-01-01",
                    end_date="2024-01-05",
                    start_time=None,
                    end_time=None,
                    duration=5,
                    time_unit="DAYS",
                    cross_midnight=False,
                    type="ABSENCE",
                    status_formatted=None,
                    absence_duration_category=None,
                ) )
        elif time_type == "PUBLIC_HOLIDAY":
            time_off_events.append(
                UpcomingTimeOff(
                    title="New Year's Day",
                    start_date="2024-01-01",
                    end_date="2024-01-01",
                    start_time=None,
                    end_time=None,
                    duration=1,
                    time_unit="DAYS",
                    cross_midnight=False,
                    type="PUBLIC_HOLIDAY",
                    status_formatted=None,
                    absence_duration_category=None,
                ))
            time_off_events.append (UpcomingTimeOff(
                    title="Christmas Day",
                    start_date="2024-12-25",
                    end_date="2024-12-25",
                    start_time=None,
                    end_time=None,
                    duration=1,
                    time_unit="DAYS",
                    cross_midnight=False,
                    type="PUBLIC_HOLIDAY",
                    status_formatted=None,
                    absence_duration_category=None,
                ))

        elif time_type == "NON_WORKING_DAY":
            time_off_events.append(
                UpcomingTimeOff(
                    title="Weekend",
                    start_date="2024-01-06",
                    end_date="2024-01-07",
                    start_time=None,
                    end_time=None,
                    duration=2,
                    time_unit="DAYS",
                    cross_midnight=False,
                    type="NON_WORKING_DAY",
                    status_formatted=None,
                    absence_duration_category=None,
                ))

    
    if (len(time_off_events)==0):
        return {"status_code": 400, "error": "Invalid values for time off types"}
    else:
        return UpcomingTimeOffResponse(time_off_events)
'''

## Define custom configurations

Specify the LLM and the provider with the required credentials

In [3]:
from altk.build_time.tool_enrichment_toolkit.core.config import PythonToolEnrichConfig
from altk.core.llm import get_llm, GenerationMode
import os


def get_llm_client_obj(model_id="mistralai/mistral-medium-2505"):
    WatsonXAIClient = get_llm("watsonx")
    client = WatsonXAIClient(
        model_id=model_id,
        api_key=os.getenv("WX_API_KEY"),
        project_id=os.getenv("WX_PROJECT_ID"),
        url=os.getenv("WX_URL", "https://us-south.ml.cloud.ibm.com"),
    )
    return client


config = PythonToolEnrichConfig(
    llm_client=get_llm_client_obj(model_id="mistralai/mistral-medium-2505"),
    gen_mode=GenerationMode.TEXT,
    enable_tool_description_enrichment=True,
    enable_tool_parameter_description_enrichment=True,
    enable_tool_return_description_enrichment=True,
    enable_tool_example_enrichment=True,
)

### Python Tool Enrichment 

In [25]:
from altk.core.toolkit import AgentPhase
from altk.build_time.tool_enrichment_toolkit.core.toolkit import (
    PythonToolEnrichBuildInput,
)
from altk.build_time.tool_enrichment_toolkit.utils.tool_enrichment import (
    PythonToolEnrichComponent,
)
import nest_asyncio

nest_asyncio.apply()


tool_enrich_input = PythonToolEnrichBuildInput(python_tool=python_tool)
tool_enrich_middleware = PythonToolEnrichComponent()
result = tool_enrich_middleware.process(
    data=tool_enrich_input, config=config, phase=AgentPhase.BUILDTIME
)
print(result.enriched_python_tool)

from enum import Enum
from typing import ClassVar, Optional, Type
import datetime
from dataclasses import dataclass
from langchain_core.tools import tool


class TimeOffTypes(Enum):
    """Represents the time off event types"""

    ABSENCE = "ABSENCE"
    PUBLIC_HOLIDAY = "PUBLIC_HOLIDAY"
    NON_WORKING_DAY = "NON_WORKING_DAY"


@dataclass
class UpcomingTimeOff:
    """Represents an upcoming time off event"""

    title: str
    start_date: str
    end_date: str
    start_time: Optional[str]
    end_time: Optional[str]
    duration: int
    time_unit: str
    cross_midnight: bool
    type: str
    status_formatted: Optional[str]
    absence_duration_category: Optional[str]

    # Schema: ClassVar[Type[Schema]] = Schema


@dataclass
class UpcomingTimeOffResponse:
    """Represents the response from getting a user's upcoming time off."""

    time_off_events: list[UpcomingTimeOff]

    # Schema: ClassVar[Type[Schema]] = Schema


@tool
def get_upcoming_time_off(
    user_id: str, start_